In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
352,I wasted 35 minutes of my life on this turkey ...,negative
59,"Recap: A band of five young American men, all ...",negative
637,I love the comics. Although I do have problems...,negative
2,The `plot' of this film contains a few holes y...,negative
205,"The show had great episodes, this is not one o...",negative


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Kaware's PC\AppData\Local\Temp\ipykernel_16024\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
352,wasted minute life turkey gave up main charact...,negative
59,recap band five young american men left platoo...,negative
637,love comic although problem fully understandin...,negative
2,plot film contains hole could drive massive tr...,negative
205,show great episode one them terrible episode h...,negative


In [9]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
352,wasted minute life turkey gave up main charact...,0
59,recap band five young american men left platoo...,0
637,love comic although problem fully understandin...,0
2,plot film contains hole could drive massive tr...,0
205,show great episode one them terrible episode h...,0


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Pratik-Kaware/automate-mlops-lifecycle.mlflow')
dagshub.init(repo_owner='Pratik-Kaware', repo_name='automate-mlops-lifecycle', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as Pratik-Kaware

Initialized MLflow to track repo "Pratik-Kaware/automate-mlops-lifecycle"

Repository Pratik-Kaware/automate-mlops-lifecycle initialized!

2025/06/12 13:07:15 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/502a7b21ba6844dd8bcf88736810501c', creation_time=1749713849241, experiment_id='0', last_update_time=1749713849241, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Logging the model in an articfact which can be serve read or deployed later
        logging.info("Saving and logging the model...")
        import joblib
        joblib.dump(model, "model.joblib")
        mlflow.log_artifact("model.joblib")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-12 13:24:18,446 - INFO - Starting MLflow run...
2025-06-12 13:24:18,854 - INFO - Logging preprocessing parameters...
2025-06-12 13:24:19,993 - INFO - Initializing Logistic Regression model...
2025-06-12 13:24:19,994 - INFO - Fitting the model...
2025-06-12 13:24:20,019 - INFO - Model training complete.
2025-06-12 13:24:20,021 - INFO - Logging model parameters...
2025-06-12 13:24:20,401 - INFO - Making predictions...
2025-06-12 13:24:20,403 - INFO - Calculating evaluation metrics...
2025-06-12 13:24:20,417 - INFO - Logging evaluation metrics...
2025-06-12 13:24:24,911 - INFO - Saving and logging the model...
2025-06-12 13:24:27,000 - INFO - Model training and logging completed in 8.15 seconds.
2025-06-12 13:24:27,001 - INFO - Accuracy: 0.6
2025-06-12 13:24:27,001 - INFO - Precision: 0.5652173913043478
2025-06-12 13:24:27,002 - INFO - Recall: 0.6610169491525424
2025-06-12 13:24:27,003 - INFO - F1 Score: 0.609375


🏃 View run carefree-donkey-112 at: https://dagshub.com/Pratik-Kaware/automate-mlops-lifecycle.mlflow/#/experiments/0/runs/59e6fb6e79404f6b8908fa2dd2e88b6c
🧪 View experiment at: https://dagshub.com/Pratik-Kaware/automate-mlops-lifecycle.mlflow/#/experiments/0
